**Set-up**


In [ ]:
%%capture
!pip install optuna


In [ ]:
# Import Packages
## for data and preprocessing
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

## for model fitting
import lightgbm as lgb
import xgboost as xgb
import sklearn.metrics as metric

## for hyperparameter optimization
import optuna

In [ ]:
train = pd.read_csv('/content/sample_data/california_housing_train.csv')
test = pd.read_csv('/content/sample_data/california_housing_test.csv')

names = train.columns

scaler = StandardScaler()
train = pd.DataFrame(scaler.fit_transform(train),columns=names)
test = pd.DataFrame(scaler.transform(test), columns=names)


X_train = train.drop(['median_house_value'],axis=1)
X_test  = test.drop(['median_house_value'],axis=1)
y_train = train.median_house_value
y_test  = test.median_house_value

# **OPTUNA**

## **General Overview**

Optuna optimizes any objective function. This objective function takes a set of arguments (e.g., hyperparameters) and returns a single value (e.g., validation score).  

In Optuna, we create a study. A Study consists of a set of Trials. A study is defined by the objective function and the hyperparameter space. 
Each trial is, thus, a single selection from the hyperparameter space for which we evaluate the objective function.

The optimization algorithm helps in intelligently picking the next trial to evaluate in a smart(er) way, until we find the optimal value.

In practice, every hyperparameter optimization exercise consist of 4 steps:

* define a function which **trains a model** and **returns the validation score**

* define the **hyperparameter space** through which the optimization algorithm can search (trials are instances/realizations of this space)

* create a **study**, which describes the optimization exercise: 
    * *Direction* : 
        * minimize: for (Root) Mean Squared Errors, minus-log-likelihood, ...
        * maximize: r2_score, auc, accuracy, precision, recall, f1_score, ...
    * *Sampler* : the chosen optimization technique **(Optimization)**
    * *Pruner* : early stopping of unpromising trials **(Steroids)**

* **optimize** the study using different trials in a smart way **(worker function)**


In [ ]:
%%script false --no-raise-error
# STEP 1 #
#========#

def train_evaluate(params):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model
    preds = model.predict(test_data,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
      
    #score = model.best_score['valid']['rmse']
    # Return the validation score
    return score

# STEP 2 #
#========#

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 30, 1),
              'num_leaves': trial.suggest_int('num_leaves', 2, 100),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0,.1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    # Train the model and return the validation score
    score = train_evaluate(params)
    # Return the validation score
    return score

# STEP 3 #
#========#

study = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.RandomSampler(),      # GridSampler, RandomSampler, CmaEsSampler, TPESampler (default), ...
    pruner = optuna.pruners.NopPruner()             # NopPruner, MedianPruner (default), SuccessiveHalvingPruner, HyperbandPruner,...
    )

# STEP 4 #
#========#

study.optimize(objective, n_trials=100)

In [ ]:
N_TRIALS = 200

## **Grid Search**

In [ ]:
%%time
%%capture

def train_evaluate(params):
    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1234)

    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      verbose_eval=params['verbose']
                      )
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              'verbose': -1,
              }
    
    score = train_evaluate(params)
    return score

search_space = {'learning_rate': [0.01, 0.10, 0.50],
              'max_depth': [1, 10, 20, 30],
              'num_leaves': [2, 10, 20, 100]}
study1 = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.GridSampler(search_space)
    )
study1.optimize(objective, n_trials=N_TRIALS)

In [ ]:
gridsearch = {'score': study1.best_value, 'params': study1.best_params}
print(gridsearch)

## **Random Search**

In [ ]:
%%time
%%capture
def train_evaluate(params):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model 
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    # Return the validation score
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, .1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'bagging_fraction':trial.suggest_uniform('bagging_fraction', 0, 1),
              'bagging_freq':trial.suggest_int('bagging_freq', 0, 10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    # Train the model and return the validation score
    score = train_evaluate(params)

    # Return the validation score
    return score

study2 = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.RandomSampler()
    )

study2.optimize(objective, n_trials=N_TRIALS)

In [ ]:
randomsearch = {'score': study2.best_value, 'params': study2.best_params}
print(randomsearch)

## **CMAES**

In [ ]:
%%time
%%capture
def train_evaluate(params):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model 
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    # Return the validation score
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, .1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'bagging_fraction':trial.suggest_uniform('bagging_fraction', 0, 1),
              'bagging_freq':trial.suggest_int('bagging_freq',0,10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,200)
    if trial.should_prune():
      raise optuna.TrialPruned()
    
    # Return the validation score
    return score

study3 = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.CmaEsSampler()
    )

study3.optimize(objective, n_trials=N_TRIALS)

In [ ]:
cmaessearch = {'score': study3.best_value, 'params': study3.best_params}
print(cmaessearch)

## **BOHB**

In [ ]:
%%time
%%capture
def train_evaluate(params):
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model 
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    # Return the validation score
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, .1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'bagging_fraction':trial.suggest_uniform('bagging_fraction', 0, 1),
              'bagging_freq':trial.suggest_int('bagging_freq',0,10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,200)
    if trial.should_prune():
      raise optuna.TrialPruned()
    
    # Return the validation score
    return score

study4 = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.TPESampler(),
    pruner = optuna.pruners.HyperbandPruner()
    )

study4.optimize(objective, n_trials=N_TRIALS)

In [ ]:
bohbsearch = {'score': study4.best_value, 'params': study4.best_params}
print(bohbsearch)

In [ ]:
pd.DataFrame([gridsearch['score'],randomsearch['score'],cmaessearch['score'],bohbsearch['score']],index=['Grid','Random','CMAES','BOHB'],columns=['RMSE'])

## Further Topics

### **Visualization**

In [ ]:
#History: 
trials_df = study4.trials_dataframe()
trials_df

In [ ]:
optuna.visualization.plot_optimization_history(study4)

In [ ]:
optuna.visualization.plot_param_importances(study4)

In [ ]:
optuna.visualization.plot_slice(study4)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study4)